In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from collinearity import SelectNonCollinear
import datetime as dt


data/airbnb_price.csv
listing_id: unique identifier of listing
price: nightly listing price in USD
nbhood_full: name of borough and neighborhood where listing is located

data/airbnb_room_type.xlsx This is an Excel file containing data on Airbnb listing descriptions and room types.
listing_id: unique identifier of listing
description: listing description
room_type: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

data/airbnb_last_review.tsv This is a TSV file containing data on Airbnb host names and review dates.
listing_id: unique identifier of listing
host_name: name of listing host
last_review: date when the listing was last reviewed

Our goals are to convert untidy data into appropriate formats to analyze, and answer key questions including:

What is the average price, per night, of an Airbnb listing in NYC?
How does the average price of an Airbnb listing, per month, compare to the private rental market?
How many adverts are for private rooms?
How do Airbnb listing prices compare across the five NYC boroughs?

# Importing the data

In [2]:
# Load airbnb_price.csv, prices
prices = pd.read_csv("airbnb_price.csv")
prices

,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"
3,5178,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,150 dollars,"Manhattan, Chinatown"
...,...,...,...
25204,36425863,129 dollars,"Manhattan, Upper East Side"
25205,36427429,45 dollars,"Queens, Flushing"
25206,36438336,235 dollars,"Staten Island, Great Kills"
25207,36442252,100 dollars,"Bronx, Mott Haven"


In [15]:
prices.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25202 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25202 non-null  int64 
 1   price        25202 non-null  int64 
 2   nbhood_full  25202 non-null  object
dtypes: int64(2), object(1)
memory usage: 787.6+ KB


In [18]:
prices.isna().sum()

listing_id     0
price          0
nbhood_full    0
dtype: int64

In [3]:
# Load airbnb_room_type.xlsx, xls
xls = pd.ExcelFile("airbnb_room_type.xlsx")


In [4]:
# Parse the first sheet from xls, room_types
room_types = xls.parse(0)

In [5]:
# Load airbnb_last_review.tsv, reviews
reviews = pd.read_csv("airbnb_last_review.tsv", sep="\t")
reviews

,listing_id,host_name,last_review
0,2595,Jennifer,May 21 2019
1,3831,LisaRoxanne,July 05 2019
2,5099,Chris,June 22 2019
3,5178,Shunichi,June 24 2019
4,5238,Ben,June 09 2019
...,...,...,...
25204,36425863,Rusaa,July 07 2019
25205,36427429,H Ai,July 07 2019
25206,36438336,Ben,July 07 2019
25207,36442252,Blaine,July 07 2019


In [17]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25209 entries, 0 to 25208
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   listing_id   25209 non-null  int64 
 1   host_name    25201 non-null  object
 2   last_review  25209 non-null  object
dtypes: int64(1), object(2)
memory usage: 591.0+ KB


In [20]:
reviews.isna().sum()

listing_id     0
host_name      8
last_review    0
dtype: int64

# Step 2. Cleaning the price column

In [6]:
# Remove whitespace and string characters from prices column
prices["price"] = prices["price"].str.replace(" dollars", "")
prices

,listing_id,price,nbhood_full
0,2595,225,"Manhattan, Midtown"
1,3831,89,"Brooklyn, Clinton Hill"
2,5099,200,"Manhattan, Murray Hill"
3,5178,79,"Manhattan, Hell's Kitchen"
4,5238,150,"Manhattan, Chinatown"
...,...,...,...
25204,36425863,129,"Manhattan, Upper East Side"
25205,36427429,45,"Queens, Flushing"
25206,36438336,235,"Staten Island, Great Kills"
25207,36442252,100,"Bronx, Mott Haven"


In [7]:
# Convert prices column to numeric datatype
prices["price"] = pd.to_numeric(prices["price"])

In [8]:
prices

,listing_id,price,nbhood_full
0,2595,225,"Manhattan, Midtown"
1,3831,89,"Brooklyn, Clinton Hill"
2,5099,200,"Manhattan, Murray Hill"
3,5178,79,"Manhattan, Hell's Kitchen"
4,5238,150,"Manhattan, Chinatown"
...,...,...,...
25204,36425863,129,"Manhattan, Upper East Side"
25205,36427429,45,"Queens, Flushing"
25206,36438336,235,"Staten Island, Great Kills"
25207,36442252,100,"Bronx, Mott Haven"


# Step 3. Calculating average price

In [11]:
# Subset prices for listings costing $0, free_listings
free_listings = prices["price"] == 0

In [12]:
# Update prices by removing all free listings from prices
prices = prices.loc[~free_listings]

In [22]:
# Calculate the average price, avg_price
avg_price = round(prices["price"].mean(), 2)
avg_price

141.82

# Step 4. Comparing costs to the private rental market


In [23]:
# Add a new column to the prices DataFrame, price_per_month
prices["price_per_month"] = prices["price"] * 365 / 12

/var/folders/hm/_kmly7bs42gf5b9mpld3lp1r0000gn/T/ipykernel_88107/2850506283.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices["price_per_month"] = prices["price"] * 365 / 12


In [24]:
prices

,listing_id,price,nbhood_full,price_per_month
0,2595,225,"Manhattan, Midtown",6843.750000
1,3831,89,"Brooklyn, Clinton Hill",2707.083333
2,5099,200,"Manhattan, Murray Hill",6083.333333
3,5178,79,"Manhattan, Hell's Kitchen",2402.916667
4,5238,150,"Manhattan, Chinatown",4562.500000
...,...,...,...,...
25204,36425863,129,"Manhattan, Upper East Side",3923.750000
25205,36427429,45,"Queens, Flushing",1368.750000
25206,36438336,235,"Staten Island, Great Kills",7147.916667
25207,36442252,100,"Bronx, Mott Haven",3041.666667


In [26]:
# Calculate average_price_per_month
average_price_per_month = round(prices["price_per_month"].mean(), 2)
difference = round((average_price_per_month - 3100),2)
difference

1213.61

# Step 5. Cleaning the room_type column


In [28]:
# Convert the room_type column to lowercase
room_types["room_type"] = room_types["room_type"].str.lower()

In [29]:
# Update the room_type column to category data type
room_types["room_type"] = room_types["room_type"].astype("category")

In [30]:
# Create the variable room_frequencies
room_frequencies = room_types["room_type"].value_counts()

# Step 6. What timeframe are we working with?


In [31]:
# Change the data type of the last_review column to datetime
reviews["last_review"] = pd.to_datetime(reviews["last_review"])

In [32]:
# Create first_reviewed, the earliest review date
first_reviewed = reviews["last_review"].dt.date.min()

In [33]:
# Create last_reviewed, the most recent review date
last_reviewed = reviews["last_review"].dt.date.max()

# Step 7. Joining the DataFrames


In [34]:
# Merge prices and room_types to create rooms_and_prices
rooms_and_prices = prices.merge(room_types, how="outer", on="listing_id")

In [35]:
# Merge rooms_and_prices with the reviews DataFrame to create airbnb_merged
airbnb_merged = rooms_and_prices.merge(reviews, how="outer", on="listing_id")

In [36]:
# Drop missing values from airbnb_merged
airbnb_merged.dropna(inplace=True)

# Step 8. Analyzing listing prices by NYC borough


In [37]:
# Extract information from the nbhood_full column and store as a new column, borough
airbnb_merged["borough"] = airbnb_merged["nbhood_full"].str.partition(",")[0]

In [38]:
# Group by borough and calculate summary statistics
boroughs = airbnb_merged.groupby("borough")["price"].agg(["sum", "mean", "median", "count"])

In [39]:
# Round boroughs to 2 decimal places, and sort by mean in descending order
boroughs = boroughs.round(2).sort_values("mean", ascending=False)
# Step 9. Price range by borough

# Step 9. Price range by borough

In [40]:
# Create labels for the price range, label_names
label_names = ["Budget", "Average", "Expensive", "Extravagant"]

In [41]:
# Create the label ranges, ranges
ranges = [0, 69, 175, 350, np.inf]

In [42]:
# Insert new column, price_range, into DataFrame
airbnb_merged["price_range"] = pd.cut(airbnb_merged["price"], bins=ranges, labels=label_names)

In [43]:
# Calculate occurence frequencies for each label, prices_by_borough
prices_by_borough = airbnb_merged.groupby(["borough", "price_range"])["price_range"].count()

# Step 10. Storing the final result


In [44]:
solution = {'avg_price':avg_price,
            'average_price_per_month': average_price_per_month,  
            'difference':difference,          
            'room_frequencies':room_frequencies, 
            'first_reviewed': first_reviewed,
            'last_reviewed': last_reviewed,
            'prices_by_borough':prices_by_borough}
print(solution)

{'avg_price': 141.82, 'average_price_per_month': 4313.61, 'difference': 1213.61, 'room_frequencies': entire home/apt    13266
private room       11356
shared room          587
Name: room_type, dtype: int64, 'first_reviewed': datetime.date(2019, 1, 1), 'last_reviewed': datetime.date(2019, 7, 9), 'prices_by_borough': borough        price_range
Bronx          Budget          381
               Average         285
               Expensive        25
               Extravagant       5
Brooklyn       Budget         3194
               Average        5532
               Expensive      1466
               Extravagant     259
Manhattan      Budget         1148
               Average        5285
               Expensive      3072
               Extravagant     810
Queens         Budget         1631
               Average        1505
               Expensive       291
               Extravagant      28
Staten Island  Budget          124
               Average         123
               Expensive  